Preprocessing for the table result to work better with the "athletes" variable

In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
data = pd.read_csv("olympic_results_new.csv")

In [ ]:
data.head(10)

,ID_results,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_position,country_name,athlete_full_name
0,1,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,['Stefania CONSTANTINI'; 'Amos MOSANER'],1,Italy,NaN
1,2,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,['Kristin SKASLIEN'; 'Magnus NEDREGOTTEN'],2,Norway,NaN
2,3,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,['Almida DE VAL'; 'Oskar ERIKSSON'],3,Sweden,NaN
3,4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Jennifer DODDS'; 'Bruce MOUAT'],4,Great Britain,NaN
4,5,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Rachel HOMAN'; 'John MORRIS'],5,Canada,NaN
5,6,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Zuzana HAJKOVA'; 'Tomas PAUL'],6,Czech Republic,NaN
6,7,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Jenny PERRET'; 'Martin RIOS'],7,Switzerland,NaN
7,8,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Vicky PERSINGER'; 'Christopher PLYS'],8,United States of America,NaN
8,9,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Suyuan FAN'; 'Zhi LING'],9,People's Republic of China,NaN
9,10,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,['Tahli GILL'; 'Dean HEWITT'],10,Australia,NaN


In [ ]:
# Extract the relevant columns for each table
table_1 = data[['athlete_full_name', 'ID_results', 'discipline_title', 'event_title', 'slug_game', 'participant_type', 'medal_type', 'rank_position', 'country_name']]
table_2 = data[['athletes', 'ID_results', 'discipline_title', 'event_title', 'slug_game', 'participant_type', 'medal_type', 'rank_position', 'country_name']]

In [ ]:
# Remove rows where 'athlete_full_name' or 'athletes' is NaN
table_1_cleaned = table_1.dropna(subset=['athlete_full_name'])
table_2_cleaned = table_2.dropna(subset=['athletes'])

In [ ]:
table_2_cleaned.head(10)

,athletes,ID_results,discipline_title,event_title,slug_game,participant_type,medal_type,rank_position,country_name
0,['Stefania CONSTANTINI'; 'Amos MOSANER'],1,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,1,Italy
1,['Kristin SKASLIEN'; 'Magnus NEDREGOTTEN'],2,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,2,Norway
2,['Almida DE VAL'; 'Oskar ERIKSSON'],3,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,3,Sweden
3,['Jennifer DODDS'; 'Bruce MOUAT'],4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,4,Great Britain
4,['Rachel HOMAN'; 'John MORRIS'],5,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,5,Canada
5,['Zuzana HAJKOVA'; 'Tomas PAUL'],6,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,6,Czech Republic
6,['Jenny PERRET'; 'Martin RIOS'],7,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,7,Switzerland
7,['Vicky PERSINGER'; 'Christopher PLYS'],8,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,8,United States of America
8,['Suyuan FAN'; 'Zhi LING'],9,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,9,People's Republic of China
9,['Tahli GILL'; 'Dean HEWITT'],10,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,10,Australia


Convert the "athletes" field into a format readable by Neo4j

In [ ]:
# Function to clean and parse the athletes field
def parse_athletes(athletes_str):
    # Replace semicolons with commas to make it a valid Python list
    athletes_str = athletes_str.replace(';', ',')
    # Convert string representation of list to actual list
    athletes_list = ast.literal_eval(athletes_str)
    # Convert list back to a Neo4j-compatible string
    return "; ".join(athletes_list)

# Apply the function to the athletes column
table_2_cleaned['athletes'] = table_2_cleaned['athletes'].apply(parse_athletes)

<ipython-input-16-0883e904b8a6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_2_cleaned['athletes'] = table_2_cleaned['athletes'].apply(parse_athletes)


In [ ]:
table_2_cleaned.to_csv('GameTeam.csv', index=False)

In [ ]:
table_1_cleaned.to_csv('IndividualGame.csv', index=False)

**Reducing the dataset** for computational reasons. \\
[at the end we didn't use it]

In [ ]:
athlete = pd.read_csv("/content/olympic_athletes_new.csv", encoding='ISO-8859-1')

In [ ]:
individual = pd.read_csv("/content/IndividualGame.csv")

In [ ]:
distinct_values = {}

for column in individual.columns:
    distinct_values[column] = individual[column].unique()

for column, values in distinct_values.items():
    print(f"Distinct values in column '{column}':")
    print(values)
    print()

In [ ]:
stratify_columns = ['discipline_title', 'event_title', 'medal_type', 'country_name']

In [ ]:
def stratified_sample(df, stratify_cols, frac, random_state=None):
    assert 0 < frac <= 1, "frac must be between 0 and 1"
    grouped = df.groupby(stratify_cols, group_keys=False)
    stratified_df = grouped.apply(lambda x: x.sample(frac=frac, random_state=random_state))

    return stratified_df.reset_index(drop=True)


# Apply stratified sampling to get 20% sample
sample_frac = 0.7
stratified_sample_df = stratified_sample(individual, stratify_columns, sample_frac, random_state=42)
print(stratified_sample_df)


In [ ]:
#to ensure we have all the hosts
len(pd.unique(stratified_sample_df["slug_game"]))

53

*Selecting the athletes corresponding to the filtered out IndividualGame*

In [ ]:
unique_athletes = pd.unique(stratified_sample_df["athlete_full_name"])
filtered_athletes = athlete[athlete["athlete_full_name"].isin(unique_athletes)]
print(filtered_athletes)

*To have more results but still ensuring that we have the same athletes that we filtered out for the athlete df*

In [ ]:
unique_athletes = pd.unique(stratified_sample_df["athlete_full_name"])
filtered_individualgame = individual[individual["athlete_full_name"].isin(unique_athletes)]
print(filtered_individualgame)

          athlete_full_name  ID_results    discipline_title  \
5               Matt GRAHAM          16    Freestyle Skiing   
6           Ikuma HORISHIMA          17    Freestyle Skiing   
7               Daichi HARA          18    Freestyle Skiing   
16         Mikael KINGSBURY          27    Freestyle Skiing   
18          Walter WALLBERG          29    Freestyle Skiing   
...                     ...         ...                 ...   
141583  Graziano MANCINELLI      149542  Equestrian Jumping   
141584        Piero D'INZEO      149543  Equestrian Jumping   
141606   Jacob Tullin THAMS      149583         Ski Jumping   
141608        Anders HAUGEN      149585         Ski Jumping   
141609        Thorleif HAUG      149586         Ski Jumping   

                       event_title      slug_game participant_type medal_type  \
5                     Men's Moguls   beijing-2022          Athlete        NaN   
6                     Men's Moguls   beijing-2022          Athlete     BRONZE   


In [ ]:
filtered_athletes.to_csv('filtered_athletes.csv', index=False)

In [ ]:
filtered_individualgame.to_csv('filtered_individualgame.csv', index=False)